<a href="https://colab.research.google.com/github/Ingaletanvi/DSBDA/blob/main/4_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider the Bangalore House Price Data. Perform following operations.
a) Find and replace null values in the data using appropriate technique.
b) Transform the ‘Size’ column to numerical values. For Example: 2 BHK to be converted as 2
c) Transform the ‘total_sqft’ column to contain numerical values on same scale. If the range is given average value of
the range to be taken.
d) Calculate and add one more column as ‘Price_Per_Sqft’
e) Remove the outliers from Price_Per_Sqft and BHK Size column if any.
f) Apply the Linear Regression model to the data and display the training and testing performance measures as Mean
 Squared Error and Accuracy

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/Banglore Housing Prices.csv")

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['location'], inplace=True)


In [ ]:
df['bath'].fillna(df['bath'].mean(),inplace= True)

In [ ]:
df['size'].fillna(value='2 BHK', inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df['BHK'] = df['size'].apply(lambda x : int(x.split(' ')[0]))

In [ ]:
df=df.drop('size',axis=1)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
import re

def convert_sqft_to_num(sqft):
  if '-' in sqft:
    tokens = sqft.split('-')
    return (float(tokens[0])+ float(tokens[1]))/2
  else:
    numeric_part = re.search(r'\d+\.\d+|\d+' , sqft).group()
    return float(numeric_part)
df['total_sqft'] = df['total_sqft'].apply(convert_sqft_to_num)

In [ ]:
df['total_sqft'].dtypes

In [ ]:
df

In [ ]:
df['Price_Per_Sqft'] = df['price'] / df['total_sqft']
df.dtypes

In [ ]:
df.head()

In [ ]:
z_score_price_sqft = np.abs((df['Price_Per_Sqft'] - df['Price_Per_Sqft'].mean()) / df['Price_Per_Sqft'].std())
z_score_bhk = np.abs((df['BHK'] - df['BHK'].mean()) / df['BHK'].std())


In [ ]:
df_no_outliers = df[(z_score_price_sqft < 3) & (z_score_bhk < 3)]

In [ ]:
df_no_outliers.head()


**LINEAR REGRESSION**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
x = df_no_outliers[['BHK', 'total_sqft', 'bath']]
y = df_no_outliers['price']

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=42)


In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score
# Training performance
train_preds = model.predict(x_train)
train_mse = mean_squared_error(y_train, train_preds)
train_r_squared = model.score(x_train, y_train)
print(f'Training Mean Squared Error: {train_mse}')
print(f'Training R-squared: {train_r_squared}')


In [ ]:
 # Testing performance
test_preds = model.predict(x_test)
test_mse = mean_squared_error(y_test, test_preds)
test_r_squared = model.score(x_test, y_test)
print(f'Testing Mean Squared Error: {test_mse}')
print(f'Testing R-squared: {test_r_squared}')
